In [1]:
import altair as alt
from vega_datasets import data
import pandas as pd
import numpy as np

import gif

In [2]:
source = data.population.url
df = pd.read_json(source)
df['gender'] = np.where(df['sex'] == 1, 'Male', 'Female')

In [3]:
pop = df.groupby(['year', 'sex'])['people'].sum().reset_index()

In [4]:
df = pd.merge(df, pop, on=['year', 'sex'])
df['pct'] = df['people_x'] / df['people_y']

In [5]:
@gif.frame
def plot(year):
    
    d = df[df['year'] == year]
    
    base = alt.Chart(d)
    color_scale = alt.Scale(domain=['Male', 'Female'], range=['#1f77b4', '#e377c2'])

    left = base.transform_filter(
        alt.datum.gender == 'Female'
    ).encode(
        y=alt.Y('age:O', axis=None),
        x=alt.X('pct',
                title=None,
                sort=alt.SortOrder('descending')),
        color=alt.Color('gender:N', scale=color_scale, legend=None)
    ).mark_bar().properties(height=300, width=230)

    middle = base.encode(
        y=alt.Y('age:O', axis=None),
        text=alt.Text('age:Q'),
        size=alt.SizeValue(10)
    ).mark_text().properties(height=300, width=30)

    right = base.transform_filter(
        alt.datum.gender == 'Male'
    ).encode(
        y=alt.Y('age:O', axis=None),
        x=alt.X('pct', title=None),
        color=alt.Color('gender:N', scale=color_scale, legend=None)
    ).mark_bar().properties(height=300, width=230)

    chart = alt.concat(left, middle, right, spacing=5)
    return chart

In [6]:
frames = []
for year in df.year.unique():
    frame = plot(year)
    frames.append(frame)

gif.save(frames, 'pyramid.gif', duration=500)